In [ ]:
# Define the parameters in a dictionary
import os
import glob
import pandas as pd
import wandb
import optuna
from optuna.storages import JournalStorage, JournalFileStorage
from functools import partial


def run_tuning(trial, out_name):
    learning_rate = trial.suggest_float("lr", 1e-5, 6e-4, log=True),
    warmup_iters = trial.suggest_int("warmup_iters", 200, 1000, log=True),
    layerwise_pe = '[4]'
    params = {
        'max_iters': 10000,
        'lr_decay_iters': 10000,
        'general_seed': 888,
        'out_dir': 'outputs',
        'pe_type': 'original',  # or 'sin'
        'use_residual': '[0,1,2,4,5]',

    }

    # Updating the out_name format

    # Updating the out_name format
    params['out_name'] = out_name

    print(f"Running with general_seed: {params['out_name']}")

    # Loop through the parameters

    
    # Construct the output directory and other variables
    output_directory = os.path.join(params['out_dir'], f"{params['out_name']}/addition_reverse_sd{params['general_seed']}_lr{learning_rate}_wu{warmup_iters}")
    wandb_run_name = f"addition_reverse_sd{params['general_seed']}_lr{learning_rate}_wu{warmup_iters}"
    wandb_project = params['out_name']

    # Construct the command using parameters from the dictionary
    command_params = {
        'use_pe': params['pe_type'],
        'use_residual': params['use_residual'],
        'max_iters': params['max_iters'],
        'lr_decay_iters': params['lr_decay_iters'],
        'layerwise_pe': layerwise_pe,
        'out_dir': output_directory,
        'wandb_run_name': wandb_run_name,
        'learning_rate': learning_rate,
        'warmup_iters': warmup_iters,
        'wandb_project': wandb_project
    }

    command = "python train.py pe_info/config2_pe/addition/reverse/jason_train_addition_bal.py"
    for key, value in command_params.items():
        command += f" --{key}={value}"

    output_files = glob.glob('../'+output_directory + '**/**/**.csv', recursive=True)
    output_files.sort(key=os.path.getctime, reverse=True)
    latest_file = output_files[0]
    df = pd.read_csv(latest_file)
    train_loss = df['train_loss'].values[-1]
    return train_loss



if __name__ == "__main__":
    out_dir = "./outputs"
    out_name = f"out3_tuning"
    storage = JournalStorage(
        JournalFileStorage(f"{out_dir}/{out_name}/tuning.log")
    )
    try:
        study = optuna.load_study(
            storage=storage,
            study_name=out_name,
        )
    except:
        study = optuna.create_study(
            direction="minimize",
            storage=storage,  # Specify the storage URL here.
            study_name=out_name,
            pruner=optuna.pruners.MedianPruner(),
            load_if_exists=True,
        )
    trial_function = partial(
        run_tuning, out_name = out_name
    )
    study.optimize(trial_function, n_trials=32)
    print(study.best_params)

In [ ]:
import json
import logging
import os

# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import sys
from functools import partial
from pathlib import Path
from typing import Optional
from io import StringIO
import traceback

import optuna
import torch
from optuna.storages import JournalStorage, JournalFileStorage
import yaml

logging.basicConfig(level=logging.INFO)

try:
    from google.colab import drive
except ImportError:
    logging.info("Local machine detected")
    sys.path.append(os.path.realpath(".."))
else:
    logging.info("Colab detected")
    drive.mount("/content/drive")
    sys.path.append("/content/drive/MyDrive/ecg-reconstruction/src")

from ecg.trainer import Trainer, TrainerConfig
from ecg.reconstructor.cnn.cnn import StackedCNN
from ecg.reconstructor.transformer.transformer import UFormer, NaiveTransformerEncoder
from ecg.reconstructor.transformer.fastformer import (
    Fastformer,
    UFastformer,
    FastformerPlus,
)
from ecg.reconstructor.lstm.lstm import LSTM, CNNLSTM
from ecg.util.device import get_device
from ecg.util.tree import deep_merge
from ecg.util.path import get_project_root_dir


def train_experiment(
    trial: optuna.Trial,
    base_config: TrainerConfig,
    tuning_dir: Path,
) -> Optional[float]:
    """
    This is the main function for optuna to tune a model.
    """
    reconstructor_type = base_config["reconstructor"]["type"]
    config = deep_merge(base_config, reconstructor_type.suggest_config(trial))

    # The followings are the configs after tuning.
    config["optimizer"]["args"] = {
        "lr": trial.suggest_float("lr", 1e-5, 6e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 5e-6, 5e-2, log=True),
    }



    config_stream = StringIO()
    yaml.dump(config, config_stream, yaml.Dumper, indent=4)
    logging.info("Config:\n%s", config_stream.getvalue())

    config_to_log = config.copy()
    del config_to_log["reconstructor"]
    logging.info(json.dumps(config_to_log, indent=4))
    trainer = Trainer(config)
    tuning_config_dir = tuning_dir / f"trial_{trial.number}"
    with tuning_config_dir.open("w", encoding="utf-8") as config_file:
        yaml.dump(trainer.config, config_file, Dumper=yaml.Dumper)
    try:
        loss = trainer.fit(trial=trial)
    except RuntimeError:
        error_stream = StringIO()
        traceback.print_exc(file=error_stream)
        logging.error("Training failed\n%s", error_stream.getvalue())
        loss = None
    del trainer
    if get_device().type == "cuda":
        torch.cuda.empty_cache()
    return loss


base_config: TrainerConfig = {
    "in_leads": [0, 1, 8],
    "out_leads": [6, 7, 9, 10, 11],
    "max_epochs": trial_epochs,
    "accumulate_grad_batches": 1,
    "dataset": {
        "common": {
            # "predicate": "lambda f: f['SB'][:]",
            "predicate": None,
            "signal_dtype": "float32",
            "filter_type": "butter",
            "filter_args": {"N": 3, "Wn": (0.5, 60), "btype": "bandpass"},
            "mean_normalization": True,
            "feature_scaling": False,
            "include_original_signal": False,
            "include_filtered_signal": False,
            "include_labels": {},
        },
        "train": {"hdf5_filename": f"{dataset_name}/train.hdf5"},
        "eval": {"hdf5_filename": f"{dataset_name}/validation.hdf5"},
    },
    "dataloader": {
        "common": {"num_workers": 6},
    },
    "reconstructor": {"type": MODEL_TYPE},
}


if __name__ == "__main__":
    tuning_dir = get_project_root_dir() / "src" / "tuning" / MODEL_TYPE.__name__
    tuning_dir.mkdir(exist_ok=True, parents=True)
    experiment_name = f"tuning_logs_{MODEL_TYPE.__name__}"
    storage = JournalStorage(
        JournalFileStorage(str(tuning_dir / f"{experiment_name}.log"))
    )
    try:
        study = optuna.load_study(
            storage=storage,
            study_name=experiment_name,
        )
    except:
        study = optuna.create_study(
            direction="minimize",
            storage=storage,  # Specify the storage URL here.
            study_name=experiment_name,
            pruner=optuna.pruners.MedianPruner(),
            load_if_exists=True,
        )
    trial_function = partial(
        train_experiment, base_config=base_config, tuning_dir=tuning_dir
    )
    study.optimize(trial_function, n_trials=32)
    best_number = study.best_trial.number
    with open(tuning_dir / "best_trial", "w") as f:
        f.write(f"{best_number}")
    print(study.best_params)